In [1]:
import math

def knnCustomer(trainingData, trainingLabel, testData, saveName):
    def fill_missing(trainingData, trainingLabel, testData):
        #traindata    
        with open(trainingData, 'r') as dataIn:
            data=dataIn.readlines()
        dataIn.close()

        #traindataLabel
        with open(trainingLabel, 'r') as dataIn:
            dataLabel=dataIn.readlines()
        dataIn.close()

        #test Data
        with open(testData, 'r') as dataIn:
            testData=dataIn.readlines()
        dataIn.close()

        #number of rows
        print("Rows Data {}".format(len(data)))
        print("Rows Label {}".format(len(dataLabel)))
        cleanData_json={}
        amountOfFeatures=0
        highest=1.00000000000000e+99

        #train Data
        for i in range(0, len(data)):
            temp_data=data[i].replace("e+00","").split("\t")
            if len(temp_data)==1:
                temp=temp_data[0]
                temp_data=temp.split()
#             print((temp_data))
            amountOfFeatures=len(temp_data)
            #hold indexes of empty values
            highest=1.00000000000000e+99
            count=0
            #find column that the empty value is in
            index_of_empty=[]
            index_with_values=[]
            for j in range(0, len(temp_data)):
#                 currVal=temp_data[j]
#                 temp_data[j]=currVal.replace("e+00","")
#train data
                
                if float(temp_data[j]) == float(highest):
                    index_of_empty.append(j)
                    count+=1
                else:
                    index_with_values.append(j)

            cleanData_json.update({i:[temp_data, count, index_of_empty, index_with_values]})

        print("Amount of features from Train{}".format(amountOfFeatures))


        #train label
        label={}
        labelOut=[]

        for things in dataLabel:
            if "\n" in things:
                newThing=things.replace("\n","")
                things=newThing
            if things not in label.keys():
                label.update({things:1})
            else:
                currVal=label[things]+1
                label.update({things: currVal})
            labelOut.append(things)

        print(label)

        TestcleanData_json={}
        amountOfFeatures=0
        #test Data
        for i in range(0, len(testData)):
            temp_data=data[i].replace("e+00","").split("\t")
            if len(temp_data)==1:
                temp=temp_data[0]
                temp_data=temp.split()
            #hold indexes of empty values
            highest=1.00000000000000e+99
            count=0
            #find column that the empty value is in
            index_of_empty=[]
            index_with_values=[]
            for j in range(0, len(temp_data)):
                currVal=temp_data[j]
                amountOfFeatures= len(temp_data)                  
                if "\n" in temp_data[j]:
                    temp=temp_data[j]
                    temp_data[j]=temp.replace("\n","")
                
                    
                if float(temp_data[j]) == float(highest):
                    index_of_empty.append(j)
                    count+=1
                else:
                    index_with_values.append(j)

            TestcleanData_json.update({i:[temp_data, count, index_of_empty, index_with_values]})


        print("Amount of features from Test {}".format(amountOfFeatures))

       #[ key for row in data : [original data , amount of empty data,[indexes of all empty values], [index with values]]  ]
        #now I have the 'clean' data but still with empty values...supposed to be set to the 1E9th


        def eucDist(primaryRow, index_withValues, iterationRow, index_NoValues):
            def dist(a,b):
                temp=float(a-b)
                return temp*temp

            sumDist =float(0)
            columnsUsed =[]
            if len(cleanData_json[primaryRow][0])!= len(cleanData_json[iterationRow][0]):
                return False
            else:
                #for each column that has values in rowA
                for index in index_withValues:

                    #if the current column is NOT null in the rowB
                    #that means this column for Row A is not null AND the this colmn is not null for Row B
                    #get euc dist
                    if index not in index_NoValues:
                        columnsUsed.append(index)
                        value_fromRowA=cleanData_json[primaryRow][0][index]
                        value_fromRowB=cleanData_json[iterationRow][0][index]
                        distance=dist(float(value_fromRowA),float(value_fromRowB) )
                        sumDist+=float(distance)
                EucDist=float(math.sqrt(sumDist))
                return [EucDist, columnsUsed]


        ##Finding KNN values for train data
        rows=list(cleanData_json.keys())
        Knn_NullValue_columns_json={}
        print("finding KNN Values FOR training Data")
        for cur_data_row in rows:
            #list of columns with values
            values=cleanData_json[cur_data_row][3]
            data_perRow={}
            #need to find euclidean distance from cur_data_row to every other row

            for knn_row in rows:
                if knn_row != cur_data_row:
                    temp_indices_of_nullData=cleanData_json[knn_row][2]


                    #returns [sumDist, columnsUsed]
                    response=eucDist(cur_data_row, values, knn_row, temp_indices_of_nullData)
                    #update the relatinship between current row and row we are comparing it to
                    data_perRow.update({knn_row: response})
            #update the data per row from every other row    
            Knn_NullValue_columns_json.update({cur_data_row: data_perRow})


        #i have all knn values of each row with what columns the value for derived each row FOR TRAIN
        # Knn_NullValue_columns_json



    ##Finding KNN values for test data
        rows=list(TestcleanData_json.keys())
        Knn_NullValue_columns_test={}
        print("filnding KNN values FOR test Data")
        for cur_data_row in rows:
            #list of columns with values
            values=cleanData_json[cur_data_row][3]
            data_perRow={}
            #need to find euclidean distance from cur_data_row to every other row

            for knn_row in rows:
                if knn_row != cur_data_row:
                    temp_indices_of_nullData=cleanData_json[knn_row][2]


                    #returns [sumDist, columnsUsed]
                    response=eucDist(cur_data_row, values, knn_row, temp_indices_of_nullData)
                    #update the relatinship between current row and row we are comparing it to
                    data_perRow.update({knn_row: response})
            #update the data per row from every other row    
            Knn_NullValue_columns_test.update({cur_data_row: data_perRow})


        #i have all knn values of each row with what columns the value for derived each row FOR TEST
        # Knn_NullValue_columns_test

        print("filling empty values FOR training Data")
        #filling IN EMPTY VALUES FOR TRAINING DATA
        s=list(Knn_NullValue_columns_json.keys())
        print("number of rows to Iterate {}".format(len(s)))
        ValuesKnned=0
        for x in range (0, len(s)):
            data_for_currentRow=Knn_NullValue_columns_json[s[x]]
            indexofNullValues=cleanData_json[s[x]][2]

            #rows that contributed
            row_Contribution=[]
        #     print(indexofNullValues)

            for emptyValuekey in indexofNullValues:
                newValue=float(0)
                KNNdata_for_currentRow=Knn_NullValue_columns_json[s[x]]
        #knn dict holds k:(k-1)
        #so keys should not be hhave k 0:123  , 1:023 , 2:013
        #         print("type of empty Value {}".format(str(type(emptyValuekey))))
                for keys in KNNdata_for_currentRow:
                    ##cleanData_json[keys][3] in a list of columns that have a value that is not 'empty'
                    if emptyValuekey in cleanData_json[keys][3]:
                        valueFromOriginal=cleanData_json[keys][0][emptyValuekey]


                        temp=row_Contribution
                        row_Contribution.append(float(valueFromOriginal))


                row_Contribution.sort()
        #         print(row_Contribution)
                knn=math.sqrt(len(row_Contribution))
                temp=float(0)

                if knn !=0:
                    ValuesKnned+=1
                    for y in range(0, int(knn)):

                        temp+=float((row_Contribution[y]))

                    newValue=float(temp/float(knn))
    #                 print("changed value :{}".format(newValue))

                    cleanData_json[s[x]][0][emptyValuekey] =newValue   
        #             #at this stage I have the rows that can contribute to the empty value
                else:
                    print("a value was not found")
            if x%50==0:
                print("Done with row {}".format(str(x)))

        print("filling empty values FOR test Data")
        #filling IN EMPTY VALUES FOR TESTING DATA
        s=list(Knn_NullValue_columns_test.keys())
        print("number of rows to Iterate {}".format(len(s)))
        ValuesKnned=0
        for x in range (0, len(s)):
            data_for_currentRow=Knn_NullValue_columns_test[s[x]]
            indexofNullValues=TestcleanData_json[s[x]][2]

            #rows that contributed
            row_Contribution=[]
        #     print(indexofNullValues)

            for emptyValuekey in indexofNullValues:
                newValue=float(0)
                KNNdata_for_currentRow=Knn_NullValue_columns_test[s[x]]
        #knn dict holds k:(k-1)
        #so keys should not be hhave k 0:123  , 1:023 , 2:013
        #         print("type of empty Value {}".format(str(type(emptyValuekey))))
                for keys in KNNdata_for_currentRow:
                    ##TestcleanData_json[keys][3] in a list of columns that have a value that is not 'empty'
                    if emptyValuekey in TestcleanData_json[keys][3]:
                        valueFromOriginal=TestcleanData_json[keys][0][emptyValuekey]


                        temp=row_Contribution
                        row_Contribution.append(float(valueFromOriginal))


                row_Contribution.sort()
        #         print(row_Contribution)
                knn=math.sqrt(len(row_Contribution))
                temp=float(0)

                if knn !=0:
                    ValuesKnned+=1
                    for y in range(0, int(knn)):

                        temp+=float((row_Contribution[y]))

                    newValue=float(temp/float(knn))
    #                 print("changed value :{}".format(newValue))

                    TestcleanData_json[s[x]][0][emptyValuekey] =newValue   
        #             #at this stage I have the rows that can contribute to the empty value
                else:
                    print("a value was not found")
            if x%15==0:
                print("Done with row {}".format(str(x)))       
        # for eachRow in cleanData_json:
        #     emptyVal=0
        #     highest=1.00000000000000e+99
        #     for numbers in cleanData_json[eachRow][0]:
        #         if float(numbers)==float(highest):
        #             emptyVal+=1
        #     print("Row {}: old amount of empty Vals {} , new amount of empty vals {}".format(str(eachRow),str(cleanData_json[eachRow][1]), str(emptyVal)))
        #     print("Value changed = {}".format(str(ValuesKnned)))

        with open("./NewNonEmptyValues.txt", 'w') as o:
            for everything in cleanData_json:
                o.write(str(list (cleanData_json[everything][0])))
        o.close()
        print("returned [cleanData_json, labelOut, TestcleanData_json]")

        return [cleanData_json, labelOut, TestcleanData_json]

    data=fill_missing(trainingData, trainingLabel, testData)
    
    def eucDist(testRow, trainRow):
    #     print(len(testRow))
        def dist(a,b):
    #         print("dif= {} - {} ".format(a,b))
            dif=a-b
    #         print(math.sqrt(dif*dif))
            return math.sqrt(dif*dif)

        sum=float(0)
    #     print(testRow)
        for x in range(0,len(testRow)):
            test=testRow[x]
            train=trainRow[x]

            distance=dist(float(train),float(test))
            sum+=float(distance)


        return sum 


    # In[4]:


    trainingData=data[0]
    trainingLabel=data[1]
    testingData=data[2]
    BookKeeper={}

    print("label is type {}".format(type(trainingLabel)))
    print(trainingLabel)
    print("training data is type {}".format(type(trainingData)))
    # print(trainingData)
    print("testing data is type {}".format(type(testingData)))
    # print(testingData)


    # In[6]:


    print(len(testingData))
    def sortFunc(e):
        for thing in e.keys():
            return e[thing]
    ###################################
    k=int(math.sqrt(len(trainingData)))
    KNNdist={}
    print("K = {}".format(k))
    print("getting KNN of training data for Test inputs")
    for thing in testingData:
        dataForTestRow=testingData[thing][0]
        KnnData=[]
        k=math.sqrt(len(trainingData))

        #we only need sqrt of the total n = k amount

        for rows in trainingData:
            #data values
            dataForTrainingRow=trainingData[rows][0]
    #         print((dataForTrainingRow))
            sum=eucDist(dataForTestRow, dataForTrainingRow)/len(trainingData)


            KnnData.append({rows:sum})

        KNNdist.update({thing:KnnData})
        
        


    ####################
    #i HAVE KNN DATA FOR TEST ON TRAINING



    # In[30]:


    def FindMax(ClosestRows, labels):
        bestClassifier={}
        for eachRow in ClosestRows:
            for key in eachRow.keys():
    #             print(eachRow[key])
                classifier=labels[key]
    #             print(classifier)

            if classifier not in bestClassifier.keys():
                bestClassifier.update({classifier:1})
            else:
                currVal=bestClassifier[classifier]+1
                bestClassifier.update({classifier:currVal})
         
        return (bestClassifier)




    # In[32]:


    k=int(math.sqrt(len(trainingData)))
    print(k)
    classifier=[]
    for values in KNNdist:
        valueBasedOffTestData=KNNdist[values]
        valueBasedOffTestData.sort(key=sortFunc)
        closestRows=valueBasedOffTestData[0:k]
    #     print(closestRows)
        best=FindMax(closestRows,trainingLabel)
        
        highest=0
        val=""
        for things in best:
            if best[things]>highest:
                highest=best[things]
                val=things
        classifier.append(val)
        
    with open(saveName, 'w') as dataIn:       
        for classes in classifier:
            dataIn.write(classes)
            dataIn.write("\n")
               
    dataIn.close()
    print("{} done".format(testData))
    


In [2]:
# def knnCustomer(trainingData, trainingLabel, testData)

# tests=[["./datasets/TrainData1.txt","./datasets/TrainLabel1.txt", "./datasets/TestData1.txt","./custom_KNN_results/Classification1.txt"],["./datasets/TrainData2.txt","./datasets/TrainLabel2.txt", "./datasets/TestData2.txt","./custom_KNN_results/Classification2.txt"],["./datasets/TrainData3.txt","./datasets/TrainLabel3.txt", "./datasets/TestData3.txt","./custom_KNN_results/Classification3.txt"],["./datasets/TrainData4.txt","./datasets/TrainLabel4.txt", "./datasets/TestData4.txt","./custom_KNN_results/Classification4.txt"],["./datasets/TrainData5.txt","./datasets/TrainLabel5.txt", "./datasets/TestData5.txt","./custom_KNN_results/Classification5.txt"],["./datasets/TrainData6.txt","./datasets/TrainLabel6.txt", "./datasets/TestData6.txt","./custom_KNN_results/Classification6.txt"]]
# for things in tests:
#     for files in things:
#         knnCustomer(files[0],files[1],files[2],files[3])

knnCustomer("./datasets/TrainData1.txt","./datasets/TrainLabel1.txt", "./datasets/TestData1.txt","./custom_KNN_results/Classification1.txt")
knnCustomer("./datasets/TrainData2.txt","./datasets/TrainLabel2.txt", "./datasets/TestData2.txt","./custom_KNN_results/Classification2.txt")
knnCustomer("./datasets/TrainData3.txt","./datasets/TrainLabel3.txt", "./datasets/TestData3.txt","./custom_KNN_results/Classification3.txt")
knnCustomer("./datasets/TrainData4.txt","./datasets/TrainLabel4.txt", "./datasets/TestData4.txt","./custom_KNN_results/Classification4.txt")
knnCustomer("./datasets/TrainData5.txt","./datasets/TrainLabel5.txt", "./datasets/TestData5.txt","./custom_KNN_results/Classification5.txt")


Rows Data 150
Rows Label 150
Amount of features from Train3312
{'1': 108, '2': 14, '4': 14, '3': 11, '5': 3}
Amount of features from Test 0
finding KNN Values FOR training Data
filnding KNN values FOR test Data
filling empty values FOR training Data
number of rows to Iterate 150
Done with row 0
Done with row 50
Done with row 100
filling empty values FOR test Data
number of rows to Iterate 53
Done with row 0
Done with row 15
Done with row 30
Done with row 45
returned [cleanData_json, labelOut, TestcleanData_json]
label is type <class 'list'>
['1', '1', '2', '1', '1', '2', '4', '1', '1', '1', '4', '1', '1', '1', '1', '1', '1', '4', '1', '1', '4', '1', '4', '1', '1', '1', '3', '3', '1', '1', '4', '1', '1', '2', '3', '1', '3', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '4', '1', '1', '1', '3', '2', '1', '1', '1', '1', '2', '1', '3', '1', '5', '1', '2', '5', '1', '3', '1', '1', '1', '2', '1', '1', '5', '3', '1', '2', '2', '1', '1', '1', '1', '1', '1

Done with row 2535
Done with row 2550
Done with row 2565
Done with row 2580
Done with row 2595
Done with row 2610
Done with row 2625
Done with row 2640
Done with row 2655
Done with row 2670
Done with row 2685
returned [cleanData_json, labelOut, TestcleanData_json]
label is type <class 'list'>
['9', '9', '9', '1', '1', '8', '1', '6', '2', '4', '1', '4', '8', '7', '4', '7', '1', '9', '8', '9', '2', '9', '8', '4', '1', '9', '7', '9', '6', '4', '6', '8', '1', '1', '1', '8', '2', '8', '9', '5', '2', '1', '8', '8', '2', '6', '8', '6', '8', '6', '4', '7', '1', '9', '3', '1', '1', '9', '8', '1', '8', '4', '8', '7', '3', '1', '1', '7', '1', '9', '3', '9', '8', '1', '6', '2', '4', '3', '8', '5', '6', '9', '2', '2', '9', '4', '3', '5', '5', '9', '8', '9', '9', '7', '8', '3', '7', '5', '1', '9', '4', '8', '7', '5', '6', '8', '1', '7', '1', '7', '6', '8', '9', '4', '5', '5', '2', '3', '6', '7', '7', '7', '1', '9', '8', '8', '1', '8', '7', '6', '8', '4', '9', '1', '8', '8', '1', '9', '1', '7', '9', 

79
./datasets/TestData3.txt done
Rows Data 2547
Rows Label 2548
Amount of features from Train112
{'1': 288, '2': 275, '3': 270, '4': 292, '5': 278, '6': 287, '7': 289, '8': 298, '9': 270, '': 1}
Amount of features from Test 0
finding KNN Values FOR training Data
filnding KNN values FOR test Data
filling empty values FOR training Data
number of rows to Iterate 2547
Done with row 0
Done with row 50
Done with row 100
Done with row 150
Done with row 200
Done with row 250
Done with row 300
Done with row 350
Done with row 400
Done with row 450
Done with row 500
Done with row 550
Done with row 600
Done with row 650
Done with row 700
Done with row 750
Done with row 800
Done with row 850
Done with row 900
Done with row 950
Done with row 1000
Done with row 1050
Done with row 1100
Done with row 1150
Done with row 1200
Done with row 1250
Done with row 1300
Done with row 1350
Done with row 1400
Done with row 1450
Done with row 1500
Done with row 1550
Done with row 1600
Done with row 1650
Done with 

50
./datasets/TestData4.txt done
Rows Data 1119
Rows Label 1120
Amount of features from Train11
{'5': 471, '6': 447, '7': 141, '4': 39, '8': 14, '3': 7, '': 1}
Amount of features from Test 0
finding KNN Values FOR training Data
filnding KNN values FOR test Data
filling empty values FOR training Data
number of rows to Iterate 1119
Done with row 0
Done with row 50
Done with row 100
Done with row 150
Done with row 200
Done with row 250
Done with row 300
Done with row 350
Done with row 400
Done with row 450
Done with row 500
Done with row 550
Done with row 600
Done with row 650
Done with row 700
Done with row 750
Done with row 800
Done with row 850
Done with row 900
Done with row 950
Done with row 1000
Done with row 1050
Done with row 1100
filling empty values FOR test Data
number of rows to Iterate 480
Done with row 0
Done with row 15
Done with row 30
Done with row 45
Done with row 60
Done with row 75
Done with row 90
Done with row 105
Done with row 120
Done with row 135
Done with row 150

filnding KNN values FOR test Data
filling empty values FOR training Data
number of rows to Iterate 612
Done with row 0
Done with row 50
Done with row 100
Done with row 150
Done with row 200
Done with row 250
Done with row 300
Done with row 350
Done with row 400
Done with row 450
Done with row 500
Done with row 550
Done with row 600
filling empty values FOR test Data
number of rows to Iterate 262
Done with row 0
Done with row 15
Done with row 30
Done with row 45
Done with row 60
Done with row 75
Done with row 90
Done with row 105
Done with row 120
Done with row 135
Done with row 150
Done with row 165
Done with row 180
Done with row 195
Done with row 210
Done with row 225
Done with row 240
Done with row 255
returned [cleanData_json, labelOut, TestcleanData_json]
label is type <class 'list'>
['   9.2500000e+05', '   2.2500000e+06', '   8.0000000e+06', '   3.5000000e+06', '   1.7500000e+06', '   1.5000000e+06', '   9.5000000e+05', '   8.4250000e+05', '   1.2500000e+06', '   9.2500000e+05',

24
./datasets/TestData6.txt done
